In [5]:
import pandas as pd

data = pd.read_csv('CEQA Documents.csv', encoding='ISO-8859-1')
pd.set_option('display.max_columns', None)

In [2]:
data['Location Parcel Number'].sample(10)

314                     3150-008-056, 057
206                                   NaN
46             3203-008-056, 3203-008-057
152    3176-005-915/6, 918/9, 921-23, 930
35       3150-024-008, 009, 019, 020, 021
180            3118-018-022, 3118-018-023
354                          3203-001-031
526                                   NaN
24            3153-017-022, -023 and -024
458                                   NaN
Name: Location Parcel Number, dtype: object

### Fix edge cases on Parcel Locations

In [6]:
import pandas as pd
import numpy as np
import re

# Sample data including new edge cases
data = {
    'Parcel Numbers': [
        '3138-019-022',
        '3109-014-051',
        '3126-017-014; 3126-016-039',
        '3128-007-015, 3128-007-024',
        np.nan,
        '3386-007-007',
        np.nan,
        '3109-019-041',
        '3135-001-050, 067',
        '3204-010-054,-055,-062,-063',
        '3203-108-006 and -108',
        '3203-018-064 thru -071',
        '3219-018-01, 02; 3219-019-12, 13, 14',
        '3204008045/047, 3204001184/195',
        '3203-108-006 and -108',
        '3176-005-915/6, 918/9, 921-23, 930',
        '3150-008-072'
    ]
}

# Create DataFrame
df = pd.DataFrame(data)

# Function to clean parcel numbers by removing hyphens
def clean_parcel(parcel):
    if pd.isna(parcel):
        return "Unknown"
    else:
        return parcel.replace('-', '')

# Improved function to handle base and suffix cases, including 'and', 'thru', semicolons, and slashes
def expand_parcel_numbers(parcel):
    if pd.isna(parcel) or not isinstance(parcel, str) or not parcel.strip():
        return ["Unknown"]

    # Replace 'and' with a comma and normalize 'thru'
    parcel = parcel.replace(' and ', ', ').replace('thru', ' thru ')

    # Split by semicolons first to handle different parcel number groups
    groups = [group.strip() for group in parcel.split(';')]

    expanded_parcels = []

    for group in groups:
        # Split by commas for multiple parcels in the same group
        parts = [p.strip() for p in group.split(',')]

        base_parcel = None

        for part in parts:
            # Handle slash-separated parcels (e.g., '3204008045/047')
            if '/' in part:
                base_parcel, suffix = part.split('/')
                base_prefix = base_parcel[:-3]  # Assuming last 3 digits are the varying part
                full_base = clean_parcel(base_parcel)
                full_suffix = clean_parcel(base_prefix + suffix.zfill(3))
                expanded_parcels.append(full_base)
                expanded_parcels.append(full_suffix)

            # Handle ranges (e.g., '3203-018-064 thru -071')
            elif 'thru' in part:
                start, end = part.split('thru')
                start = start.strip()
                end = end.strip().strip('-')

                if base_parcel is None:
                    base_parcel = start  # Base is the starting parcel number

                base_prefix = '-'.join(base_parcel.split('-')[:-1]) + '-'

                if start.count('-') < 2:
                    start = base_prefix + start.strip('-')
                if end.count('-') < 2:
                    end = base_prefix + end.strip('-')

                start_number = int(start.split('-')[-1])
                end_number = int(end.split('-')[-1])

                for i in range(start_number, end_number + 1):
                    full_parcel = clean_parcel(base_prefix + f"{i:03d}")
                    expanded_parcels.append(full_parcel)

                base_parcel = start  # Update base parcel for potential further processing

            elif part.count('-') == 2:  # Full parcel number (e.g., '3219-018-01')
                base_parcel = part
                expanded_parcels.append(clean_parcel(base_parcel))

            else:  # Handle comma-separated extensions (e.g., '02' from '3219-018-01')
                if base_parcel:
                    base_prefix = '-'.join(base_parcel.split('-')[:-1]) + '-'
                    full_parcel = clean_parcel(base_prefix + part.strip('-'))
                    expanded_parcels.append(full_parcel)
                else:
                    return ["Unknown"]

    return expanded_parcels

# Apply the function to split and expand parcel numbers
df['Expanded Parcels'] = df['Parcel Numbers'].apply(expand_parcel_numbers)

# Find the maximum number of parcels in any row
max_columns = df['Expanded Parcels'].apply(len).max()

# Create the new columns dynamically based on the max number of parcels
parcel_columns = pd.DataFrame(df['Expanded Parcels'].to_list(), columns=[f'Parcel_{i+1}' for i in range(max_columns)])

# Replace any NaN or None values in the DataFrame with "Unknown"
parcel_columns.fillna("Unknown", inplace=True)

# Function to validate the parcels based on length and content
def validate_parcel(parcel):
    # Check if the parcel is a string and consists of exactly 9-10 digits
    if isinstance(parcel, str) and re.match(r'^\d{9,10}$', parcel):
        return parcel
    else:
        return "Unknown"

# Apply the validation to all parcel columns
for col in parcel_columns.columns:
    parcel_columns[col] = parcel_columns[col].apply(validate_parcel)

# Concatenate the new columns back to the original DataFrame
df = pd.concat([df, parcel_columns], axis=1)

# Drop the 'Expanded Parcels' and original 'Parcel Numbers' column (optional)
df.drop(['Expanded Parcels', 'Parcel Numbers'], axis=1, inplace=True)

# Display the cleaned DataFrame
print(df)


      Parcel_1    Parcel_2    Parcel_3    Parcel_4    Parcel_5    Parcel_6  \
0   3138019022     Unknown     Unknown     Unknown     Unknown     Unknown   
1   3109014051     Unknown     Unknown     Unknown     Unknown     Unknown   
2   3126017014  3126016039     Unknown     Unknown     Unknown     Unknown   
3   3128007015  3128007024     Unknown     Unknown     Unknown     Unknown   
4      Unknown     Unknown     Unknown     Unknown     Unknown     Unknown   
5   3386007007     Unknown     Unknown     Unknown     Unknown     Unknown   
6      Unknown     Unknown     Unknown     Unknown     Unknown     Unknown   
7   3109019041     Unknown     Unknown     Unknown     Unknown     Unknown   
8   3135001050  3135001067     Unknown     Unknown     Unknown     Unknown   
9   3204010054  3204010055  3204010062  3204010063     Unknown     Unknown   
10  3203108006  3203108108     Unknown     Unknown     Unknown     Unknown   
11  3203018064  3203018065  3203018066  3203018067  3203018068  

In [20]:
import pandas as pd
import numpy as np
import re

# Function to clean parcel numbers by removing hyphens, non-numeric characters, and handling NaN values
def clean_parcel(parcel):
    """Clean parcel numbers by removing hyphens, non-numeric characters, and handling NaN values."""
    if pd.isna(parcel):
        return "Unknown"
    else:
        return re.sub(r'\D', '', parcel)  # Remove all non-numeric characters

# Function to validate the parcels based on length and content
def validate_parcel(parcel):
    """Validate the parcel based on length and content."""
    # Check if the parcel is a string and consists of exactly 9-10 digits
    if isinstance(parcel, str) and re.match(r'^\d{9,10}$', parcel):
        return parcel
    else:
        return "Unknown"

# Function to expand parcel numbers by handling complex cases like 'and', 'thru', semicolons, slashes, and other edge cases
def expand_parcel_numbers(parcel):
    """Expand parcel numbers by handling complex cases like 'and', 'thru', semicolons, slashes, and new edge cases."""
    if pd.isna(parcel) or not isinstance(parcel, str) or not parcel.strip():
        return ["Unknown"]

    # Replace 'and' with a comma and normalize 'thru'
    parcel = parcel.replace(' and ', ', ').replace('thru', ' thru ').replace(' to ', ' to ').replace('&', ', ')

    # Handle multiple semicolons or commas
    parcel = re.sub(r'[;,]+', ',', parcel)  # Replace multiple semicolons or commas with a single comma

    # Split by semicolons first to handle different parcel number groups
    groups = [group.strip() for group in parcel.split(';')]

    expanded_parcels = []

    for group in groups:
        # Split by commas for multiple parcels in the same group
        parts = [p.strip() for p in group.split(',')]

        base_parcel = None

        for part in parts:
            # Check if the part contains slashes (e.g., '3204008045/047') and handle accordingly
            if '/' in part:
                split_parts = part.split('/')
                base_parcel = split_parts[0]
                if len(split_parts) > 1:
                    for suffix in split_parts[1:]:
                        if suffix.isdigit():
                            base_prefix = base_parcel[:-len(suffix)]  # Adjust the prefix based on suffix length
                            expanded_parcels.append(validate_parcel(clean_parcel(base_prefix + suffix.zfill(3))))
                        else:
                            expanded_parcels.append("Unknown")
                    else:
                        expanded_parcels.append(validate_parcel(clean_parcel(base_parcel)))

            # Handle ranges with 'to' (e.g., '3123-014-900 to 916')
            elif ' to ' in part:
                try:
                    start, end = part.split(' to ')
                    start = start.strip()
                    end = end.strip()

                    if base_parcel is None:
                        base_parcel = start  # Base is the starting parcel number

                    base_prefix = '-'.join(base_parcel.split('-')[:-1]) + '-'

                    start_number = int(start.split('-')[-1])
                    end_number = int(end)

                    # Expand the range and append the parcels
                    for i in range(start_number, end_number + 1):
                        expanded_parcels.append(validate_parcel(clean_parcel(base_prefix + f"{i:03d}")))
                except Exception:
                    expanded_parcels.append("Unknown")  # Handle invalid range format

            # Handle ranges with 'thru' (e.g., '3203-018-064 thru -071')
            elif 'thru' in part:
                try:
                    start, end = part.split('thru')
                    start = start.strip()
                    end = end.strip().strip('-')

                    if base_parcel is None:
                        base_parcel = start  # Base is the starting parcel number

                    base_prefix = '-'.join(base_parcel.split('-')[:-1]) + '-'

                    start_number = int(start.split('-')[-1])
                    end_number = int(end.split('-')[-1])

                    # Expand the range and append the parcels
                    for i in range(start_number, end_number + 1):
                        expanded_parcels.append(validate_parcel(clean_parcel(base_prefix + f"{i:03d}")))
                except Exception:
                    expanded_parcels.append("Unknown")  # Handle invalid range format

            # Handle full parcel numbers (e.g., '3219-018-01')
            elif part.count('-') == 2:
                base_parcel = part
                expanded_parcels.append(validate_parcel(clean_parcel(base_parcel)))

            # Handle comma-separated extensions (e.g., '02' from '3219-018-01')
            elif base_parcel:
                base_prefix = '-'.join(base_parcel.split('-')[:-1]) + '-'
                full_parcel = validate_parcel(clean_parcel(base_prefix + part.strip('-')))
                expanded_parcels.append(full_parcel)

            else:
                expanded_parcels.append("Unknown")  # Handle cases where no base parcel is found

    return expanded_parcels

# Function to process the DataFrame and expand the parcel numbers
def process_parcel_data(df):
    """Process the DataFrame and expand the parcel numbers."""
    if 'Location Parcel Number' not in df.columns:
        raise ValueError("Column 'Location Parcel Number' is missing in the DataFrame.")

    # Apply the function to expand parcel numbers
    df['Expanded Parcels'] = df['Location Parcel Number'].apply(expand_parcel_numbers)

    # Find the maximum number of parcels in any row
    max_columns = df['Expanded Parcels'].apply(len).max()

    # Create the new columns dynamically based on the max number of parcels
    parcel_columns = pd.DataFrame(df['Expanded Parcels'].to_list(), columns=[f'Parcel_{i+1}' for i in range(max_columns)])

    # Replace any NaN or None values in the DataFrame with "Unknown"
    parcel_columns.fillna("Unknown", inplace=True)

    # Concatenate the new columns back to the original DataFrame
    df = pd.concat([df, parcel_columns], axis=1)

    # Drop the 'Expanded Parcels' and original 'Location Parcel Number' column
    #df.drop(['Expanded Parcels', 'Location Parcel Number'], axis=1, inplace=True)

    df = df.drop(columns=['Expanded Parcels', 'Location Parcel Number'])  # Drop the columns
    # combine_parcels(df)  # Call the function to combine the parcel columns into a single column



    return df

# Function to combine parcel columns into a single 'Location Parcel Number' column
def combine_parcels(df, max_length=50):
    parcel_columns = [col for col in df.columns if col.startswith('Parcel_')]
    df['Location Parcel Number'] = df[parcel_columns].apply(
        lambda row: ', '.join(row[row != 'Unknown'].astype(str)), axis=1)

    # Truncate if necessary
    df['Location Parcel Number'] = df['Location Parcel Number'].apply(
        lambda val: val[:max_length] if len(val) > max_length else val
    )
    
    # replace any None values with 'Unknown' and empty strings with 'Unknown'
    df['Location Parcel Number'] = df['Location Parcel Number'].replace({None: 'Unknown'})
    df['Location Parcel Number'] = df['Location Parcel Number'].replace({'': 'Unknown'})


    # Drop extra Parcel columns
    df.drop(parcel_columns, axis=1, inplace=True)
    
    return df

# Function to execute the full process on a given DataFrame
def run_parcel_expansion(data):
    """Main function to run the parcel expansion process."""
    # Ensure data contains 'SCH Number' and 'Location Parcel Number'
    if data.empty:
        raise ValueError("Data is empty.")

    if not set(['SCH Number', 'Location Parcel Number']).issubset(data.columns):
        raise ValueError("Data must contain 'SCH Number' and 'Location Parcel Number' columns")

    # Process the DataFrame to expand parcel numbers
    df_cleaned = process_parcel_data(data)
    
    # Combine the expanded parcels back into a single column
    df_cleaned = combine_parcels(df_cleaned)

    return df_cleaned


In [21]:
# Sample data including new edge cases
data = {
    'SCH Number': [
        'SCH001', 'SCH002', 'SCH003', 'SCH004', 'SCH005', 'SCH006', 'SCH007', 'SCH008', 
        'SCH009', 'SCH010', 'SCH011', 'SCH012', 'SCH013', 'SCH014', 'SCH015', 'SCH016', 'SCH017'
    ],
    'Location Parcel Number': [
        '3138-019-022',
        '3109-014-051',
        '3126-017-014; 3126-016-039',
        '3128-007-015, 3128-007-024',
        None,
        '3386-007-007',
        None,
        '3109-019-041',
        '3135-001-050, 067',
        '3204-010-054,-055,-062,-063',
        '3203-108-006 and -108',
        '3203-018-064 thru -071',
        '3219-018-01, 02; 3219-019-12, 13, 14',
        '3204008045/047, 3204001184/195',
        '3203-108-006 and -108',
        '3176-005-915/6, 918/9, 921-23, 930',
        '3150-008-072'
    ]
}

# Convert the dictionary to a DataFrame
df = pd.DataFrame(data)

# Now you can pass the DataFrame to the run_parcel_expansion function
expanded_df = run_parcel_expansion(df)

# Print the expanded dataframe
print(expanded_df)

   SCH Number                             Location Parcel Number
0      SCH001                                         3138019022
1      SCH002                                         3109014051
2      SCH003                             3126017014, 3126016039
3      SCH004                             3128007015, 3128007024
4      SCH005                                            Unknown
5      SCH006                                         3386007007
6      SCH007                                            Unknown
7      SCH008                                         3109019041
8      SCH009                             3135001050, 3135001067
9      SCH010     3204010054, 3204010055, 3204010062, 3204010063
10     SCH011                             3203108006, 3203108108
11     SCH012  3203018064, 3203018065, 3203018066, 3203018067...
12     SCH013  321901801, 321901802, 321901912, 321901913, 32...
13     SCH014     3204008047, 3204008045, 3204001195, 3204001184
14     SCH015            

In [32]:
# find the max character length of a row in the 'Location Parcel Number' column
max_length = expanded_df['Location Parcel Number'].str.len().max()
print(f"Max character length: {max_length}")

Max character length: 50


In [ ]:
import pandas as pd

data = pd.read_csv('CEQA Documents.csv', encoding='ISO-8859-1')
pd.set_option('display.max_columns', None)

df = data[['SCH Number', 'Location Parcel Number']]
edge_cases = [2018101053, 2000081119, 2018041030]

# Show all duplicated values in the 'SCH Number' column
duplicates = data[data.duplicated(subset=['SCH Number'], keep=False)].sort_values('SCH Number')
duplicates['Received']

## Test solution with entire document

In [22]:
data = pd.read_csv('CEQA Documents.csv', encoding='ISO-8859-1')
pd.set_option('display.max_columns', None)

In [24]:
# Now you can pass the DataFrame to the run_parcel_expansion function
expanded_df = run_parcel_expansion(data)

# Print the expanded dataframe
expanded_df.sample(1)

,SCH Number,Lead Agency Name,Lead Agency Title,Lead Agency Acronym,Document Title,Document Type,Received,Posted,Document Description,Document Portal URL,Project Title,Contact Full Name,Contact Authority,Contact Job Title,Contact Email Address,Contact Address 1,Contact Address 2,Contact City,Contact State,Contact Zip Code,Contact Phone Number,Location Coordinates,Cities,Counties,County Clerks,Location Cross Streets,Location Zip Code,Location Total Acres,Location State Highways,Location Waterways,Location Airports,NOC Has Non Late Comment,NOC State Review Start Date,NOC State Review End Date,NOC Development Type,NOC Local Action,NOC Project Issues,NOC Public Review Start Date,NOC Public Review End Date,NOE Exempt Status,NOE Exempt Citation,NOE Reasons for Exemption,NOD Agency,NOD Approved By Lead Agency,NOD Approved Date,NOD Significant Environmental Impact,NOD Environmental Impact Report Prepared,NOD Negative Declaration Prepared,NOD Other Document Type,NOD Mitigation Measures,NOD Mitigation Reporting Or Monitoring Plan,NOD Statement Of Overriding Considerations Adopted,NOD Findings Made Pursuant,NOD Final EIR Available Location,Location Parcel Number
297,2007021086,"Lancaster, City of",City of Lancaster,NaN,30th Street West and Avenue K Projects,EIR,8/21/2007,NaN,The proposed projects involve the development ...,https://ceqanet.opr.ca.gov/2007021086/2,30th Street West and Avenue K Projects,Jocelyn Swain,City of Lancaster,NaN,NaN,44933 Fern Avenue,NaN,Lancaster,CA,93534,6617236249,NaN,Lancaster,Los Angeles,NaN,30th Street West and Avenue K,93536,NaN,14,NaN,NaN,Yes,8/21/2007,10/4/2007,"Commercial, Residential","General Plan Amendment, Rezone","Aesthetics, Air Quality, Biological Resources,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"3112008003, 3112001902"


### Make  Unique Primary KEY with SCH Number and Receiver Date

In [25]:
data = pd.read_csv('CEQA Documents.csv', encoding='ISO-8859-1')
pd.set_option('display.max_columns', None)

In [26]:
def split_received_date(df, column):
    """
    Process a DataFrame to split the 'received' date column into 'received_month', 
    'received_day', and 'received_year' columns.

    Args:
        df (pd.DataFrame): Input DataFrame containing the 'received' date column.
    
    Returns:
        pd.DataFrame: DataFrame with 'received_month', 'received_day', and 'received_year' columns.
    """
    
    # Check if the 'received' column exists in the DataFrame
    if column not in df.columns:
        raise ValueError("The DataFrame does not contain a 'received' column.")
    
    # Convert the 'received' column to datetime format (it handles invalid parsing automatically)
    df[column] = pd.to_datetime(df[column], format='%m/%d/%Y', errors='coerce')

    # Check for any rows that couldn't be converted to datetime (NaT)
    if df[column].isna().any():
        raise ValueError("Some dates in the 'received' column could not be parsed.")
    
    # Create new columns for month, day, and year by extracting from the datetime column
    df[f'{column}_month'] = df[column].dt.month
    df[f'{column}_day'] = df[column].dt.day
    df[f'{column}_year'] = df[column].dt.year

    # Optionally, drop the original 'received' column if no longer needed
    # df.drop('received', axis=1, inplace=True)

    return df

In [11]:
# Example DataFrame
data = {
    'Received': ['11/04/1982', '05/12/1999', '12/31/2020']
}

df = pd.DataFrame(data)

# Apply the function
df_processed = split_received_date(df, 'Received')

# Display the result
print(df_processed)

    Received  Received_month  Received_day  Received_year
0 1982-11-04              11             4           1982
1 1999-05-12               5            12           1999
2 2020-12-31              12            31           2020


In [27]:
data = pd.read_csv('CEQA Documents.csv', encoding='ISO-8859-1')
pd.set_option('display.max_columns', None)

# Split the 'Received' date column into 'Received_Month', 'Received_Day', and 'Received_Year'
data_processed = split_received_date(data, 'Received')
data_prcoessed = run_parcel_expansion(data)
# Display the processed DataFrame

data_prcoessed.sample(1)

,SCH Number,Lead Agency Name,Lead Agency Title,Lead Agency Acronym,Document Title,Document Type,Received,Posted,Document Description,Document Portal URL,Project Title,Contact Full Name,Contact Authority,Contact Job Title,Contact Email Address,Contact Address 1,Contact Address 2,Contact City,Contact State,Contact Zip Code,Contact Phone Number,Location Coordinates,Cities,Counties,County Clerks,Location Cross Streets,Location Zip Code,Location Total Acres,Location State Highways,Location Waterways,Location Airports,NOC Has Non Late Comment,NOC State Review Start Date,NOC State Review End Date,NOC Development Type,NOC Local Action,NOC Project Issues,NOC Public Review Start Date,NOC Public Review End Date,NOE Exempt Status,NOE Exempt Citation,NOE Reasons for Exemption,NOD Agency,NOD Approved By Lead Agency,NOD Approved Date,NOD Significant Environmental Impact,NOD Environmental Impact Report Prepared,NOD Negative Declaration Prepared,NOD Other Document Type,NOD Mitigation Measures,NOD Mitigation Reporting Or Monitoring Plan,NOD Statement Of Overriding Considerations Adopted,NOD Findings Made Pursuant,NOD Final EIR Available Location,Received_month,Received_day,Received_year,Location Parcel Number
344,2006081045,"Lancaster, City of",City of Lancaster,NaN,Tentative Tract Map 060367,MND,2006-08-08,NaN,Subdivision of 128 single family residences on...,https://ceqanet.opr.ca.gov/2006081045,Tentative Tract Map 060367,Jocelyn Swain,City of Lancaster,NaN,NaN,44933 N. Fern Avenue,NaN,Lancaster,CA,93534,6617236249,NaN,Lancaster,Los Angeles,NaN,37th Street East and Lancaster Boulevard,93535,30,NaN,NaN,NaN,No,8/8/2006,9/6/2006,Residential,"Land Division (Subdivision, etc.)","Biological Resources, Hazards & Hazardous Mate...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,8,2006,"3150021019, 3150021020, 3150021022, 3150021025"


In [29]:
# components are strings before concatenation
data_prcoessed.loc[:, 'entry_id'] = (
    data_prcoessed['SCH Number'].astype(str) +  # Convert SCH Number to string
    data_prcoessed['Received_month'].astype(str) +  # Convert month to string
    data_prcoessed['Received_day'].astype(str) +  # Convert day to string
    data_prcoessed['Received_year'].astype(str)  # Convert year to string
)       



# count all columns in data processed
data_prcoessed.count()


SCH Number                                            546
Lead Agency Name                                      546
Lead Agency Title                                     546
Lead Agency Acronym                                     0
Document Title                                        546
Document Type                                         546
Received                                              546
Posted                                                116
Document Description                                  544
Document Portal URL                                   546
Project Title                                         546
Contact Full Name                                     546
Contact Authority                                     530
Contact Job Title                                     111
Contact Email Address                                 139
Contact Address  1                                    440
Contact Address  2                                      7
Contact City  

In [146]:
# Get the edge cases from SCH Number
edge_cases = data[data['SCH Number'].isin(edge_cases)]

,SCH Number,Lead Agency Name,Lead Agency Title,Lead Agency Acronym,Document Title,Document Type,Received,Posted,Document Description,Document Portal URL,Project Title,Contact Full Name,Contact Authority,Contact Job Title,Contact Email Address,Contact Address 1,Contact Address 2,Contact City,Contact State,Contact Zip Code,Contact Phone Number,Location Coordinates,Cities,Counties,County Clerks,Location Cross Streets,Location Zip Code,Location Total Acres,Location Parcel Number,Location State Highways,Location Waterways,Location Airports,NOC Has Non Late Comment,NOC State Review Start Date,NOC State Review End Date,NOC Development Type,NOC Local Action,NOC Project Issues,NOC Public Review Start Date,NOC Public Review End Date,NOE Exempt Status,NOE Exempt Citation,NOE Reasons for Exemption,NOD Agency,NOD Approved By Lead Agency,NOD Approved Date,NOD Significant Environmental Impact,NOD Environmental Impact Report Prepared,NOD Negative Declaration Prepared,NOD Other Document Type,NOD Mitigation Measures,NOD Mitigation Reporting Or Monitoring Plan,NOD Statement Of Overriding Considerations Adopted,NOD Findings Made Pursuant,NOD Final EIR Available Location
141,2018101053,"Lancaster, City of",City of Lancaster,NaN,TTM No. 82039/CUP 18-08,MND,1/23/2019,NaN,The proposed project involves the creation of ...,https://ceqanet.opr.ca.gov/2018101053/2,TTM No. 82039/CUP 18-08,Jocelyn Swain,City of Lancaster,NaN,NaN,44933 N. Fern Avenue,NaN,Lancaster,CA,93534,6617236100,NaN,Lancaster,Los Angeles,NaN,15th St W and Avenue J,93534,9.92,"3123-013-900/1, 3123-014-900 to 916",14,Amargosa Creek,NaN,No,1/23/2019,2/21/2019,Other,"Land Division (Subdivision, etc.), Use Permit","Air Quality, Cultural Resources, Cumulative Ef...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
143,2018101053,"Lancaster, City of",City of Lancaster,NaN,TTM No. 82039/CUP 18-08,MND,10/22/2018,NaN,The proposed project involves the creation of ...,https://ceqanet.opr.ca.gov/2018101053,TTM No. 82039/CUP 18-08,Jocelyn Swain,City of Lancaster,NaN,NaN,44933 N. Fern Avenue,NaN,Lancaster,CA,93534,6617236100,NaN,Lancaster,Los Angeles,NaN,15th St W and Avenue J,93534,9.92,"3123-013-900/1, 3123-014-900 to 916",14,Amargosa Creek,NaN,No,10/22/2018,11/20/2018,Other,"Land Division (Subdivision, etc.), Use Permit","Aesthetics, Air Quality, Cultural Resources, C...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
152,2018041030,"Lancaster, City of",City of Lancaster,NaN,"Tentative Tract Map 78213, Conditional Use Per...",MND,4/11/2018,NaN,The project consists of a residential planned ...,https://ceqanet.opr.ca.gov/2018041030,"Tentative Tract Map 78213, Conditional Use Per...",Chris Aune,City of Lancaster,NaN,NaN,44933 N. Fern Avenue,NaN,Lancaster,CA,93534,6617236100,NaN,Lancaster,Los Angeles,NaN,"Division St, Avenue I",93534,38.89,"3176-005-915/6, 918/9, 921-23, 930",14,NaN,NaN,No,4/11/2018,5/10/2018,"Other, Residential","General Plan Amendment, Land Division (Subdivi...","Air Quality, Cultural Resources, Cumulative Ef...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
396,2000081119,"Lancaster, City of",City of Lancaster,NaN,Westview Estates,EIR,10/3/2002,NaN,"The site would be a private, gated residential...",https://ceqanet.opr.ca.gov/2000081119/2,Westview Estates,Brian Ludicke,City of Lancaster,NaN,NaN,44933 North Fern Avenue,NaN,Lancaster,CA,93534,6617236105,NaN,Lancaster,Los Angeles,NaN,Avenue I & 90th Street West,NaN,115.64,3203-001-003&004,NaN,NaN,NaN,Yes,10/3/2002,11/18/2002,Residential,"Land Division (Subdivision, etc.)","Aesthetics, Agriculture and Forestry Resources...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
407,2000081119,"Lancaster, City of",City of Lancaster,NaN,Westview Estates,NOP,8/29/2000,NaN,"The site would be a private, gated residential...",https://ceqanet.opr.ca.gov/2000081119,Westview Estates,Brian Ludicke,City of Lancaster,NaN,NaN,44933 North Fern Avenue,NaN,Lancaster,CA,93534,66

### Mock Run the Entire Code Folder

#### utils

In [32]:
import os
import requests
import pandas as pd
import numpy as np
import logging
from azure.storage.blob import BlobServiceClient
from azure.storage.blob import BlobClient
from dotenv import load_dotenv
import psycopg2
from datetime import datetime
from psycopg2 import sql
from azure.core.exceptions import ResourceNotFoundError, ResourceExistsError
from azure.storage.blob import BlobServiceClient

# Load environment variables from .env file
load_dotenv()

# Configure logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

def db_connection():
    """
    Establish a connection to the PostgreSQL database.
    
    Returns:
        connection: A psycopg2 connection object.
    """
    try:
        db_host = os.getenv('DB_HOST')
        db_name = os.getenv('DB_NAME')
        db_user = os.getenv('DB_USER')
        db_password = os.getenv('DB_PASSWORD')
        db_port = os.getenv('DB_PORT')

        logging.debug(f"Connecting to database {db_name} on host {db_host}")
        connection = psycopg2.connect(
            host=db_host,
            dbname=db_name,
            user=db_user,
            password=db_password,
            port=db_port
        )
        logging.info("Database connection established successfully")
        return connection
    except Exception as e:
        logging.error(f"Error connecting to database: {e}")
        raise

def download_csv(city_name, base_url):
    """
    Download a CSV file for a specified city.
    
    Args:
        city_name (str): The name of the city for which to download the data.
        base_url (str): The base URL for CEQA data.
    
    Returns:
        str: The local path to the downloaded CSV file.
    """
    try:
        city_query = city_name.replace(" ", "%20").replace(",", "%2C")
        download_url = f"{base_url}{city_query}&OutputFormat=CSV"
        logging.debug(f"Downloading CSV from URL: {download_url}")
        response = requests.get(download_url)
        
        # Filename for the city CSV
        file_name = f"{city_name}_ceqa.csv"
        
        # Saving the file locally temporarily before uploading to Azure
        with open(file_name, 'wb') as file:
            file.write(response.content)
        logging.info(f"CSV downloaded and saved locally as {file_name}")
        
        return file_name
    except Exception as e:
        logging.error(f"Error downloading CSV for {city_name}: {e}")
        raise

def upload_to_blob(file_name):
    """
    Upload a file to Azure Blob Storage.

    Args:
        file_name (str): The name of the file to upload.
    """
    try:
        connection_string = os.getenv('AZURE_STORAGE_CONNECTION_STRING')
        container_name = os.getenv('AZURE_CONTAINER_NAME')

        blob_service_client = BlobServiceClient.from_connection_string(connection_string)
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=file_name)

        logging.debug(f"Uploading {file_name} to Azure Blob Storage in container {container_name}")
        with open(file_name, "rb") as data:
            blob_client.upload_blob(data, overwrite=True)
        logging.info(f"Uploaded {file_name} to Azure Blob Storage.")
    
    except Exception as e:
        logging.error(f"Error uploading {file_name} to Azure Blob Storage: {e}")
        raise

def download_from_blob(file_name):
    """
    Download a file from Azure Blob Storage to a local file.

    Args:
        file_name (str): The name of the file in Azure Blob Storage.
    """
    try:
        connection_string = os.getenv('AZURE_STORAGE_CONNECTION_STRING')
        container_name = os.getenv('AZURE_CONTAINER_NAME')

        blob_service_client = BlobServiceClient.from_connection_string(connection_string)
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=file_name)

        logging.debug(f"Downloading {file_name} from Azure Blob Storage")
        download_path = f"downloaded_{file_name}"

        with open(download_path, "wb") as download_file:
            download_file.write(blob_client.download_blob().readall())

        logging.info(f"Downloaded {file_name} from Azure Blob Storage to {download_path}")
        return download_path
    except Exception as e:
        logging.error(f"Error downloading {file_name} from Azure Blob Storage: {e}")
        raise

def delete_from_blob(file_name):
    """
    Delete a file from Azure Blob Storage.

    Args:
        file_name (str): The name of the file to delete.
    """
    try:
        connection_string = os.getenv('AZURE_STORAGE_CONNECTION_STRING')
        container_name = os.getenv('AZURE_CONTAINER_NAME')

        blob_service_client = BlobServiceClient.from_connection_string(connection_string)
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=file_name)

        logging.debug(f"Deleting {file_name} from Azure Blob Storage in container {container_name}")
        blob_client.delete_blob()
        logging.info(f"Deleted {file_name} from Azure Blob Storage.")
    
    except Exception as e:
        logging.error(f"Error deleting {file_name} from Azure Blob Storage: {e}")
        raise

def cleanup_local_file(file_name):
    """
    Remove the local file once processing is done.
    
    Args:
        file_name (str): The name of the local file to be removed.
    """
    try:
        if os.path.exists(file_name):
            os.remove(file_name)
            logging.info(f"Removed local file {file_name}")
        else:
            logging.warning(f"Local file {file_name} does not exist")
    except Exception as e:
        logging.error(f"Error removing local file {file_name}: {e}")
        raise

def split_received_date(df, column):
    """
    Process a DataFrame to split the 'received' date column into 'received_month', 
    'received_day', and 'received_year' columns.

    Args:
        df (pd.DataFrame): Input DataFrame containing the 'received' date column.
    
    Returns:
        pd.DataFrame: DataFrame with 'received_month', 'received_day', and 'received_year' columns.
    """
    
    # Check if the 'received' column exists in the DataFrame
    if column not in df.columns:
        raise ValueError("The DataFrame does not contain a 'received' column.")
    
    # Convert the 'received' column to datetime format (it handles invalid parsing automatically)
    df[column] = pd.to_datetime(df[column], format='%m/%d/%Y', errors='coerce')

    # Check for any rows that couldn't be converted to datetime (NaT)
    if df[column].isna().any():
        raise ValueError("Some dates in the 'received' column could not be parsed.")
    
    # Create new columns for month, day, and year by extracting from the datetime column
    df[f'{column}_month'] = df[column].dt.month
    df[f'{column}_day'] = df[column].dt.day
    df[f'{column}_year'] = df[column].dt.year

    # Optionally, drop the original 'received' column if no longer needed
    # df.drop('received', axis=1, inplace=True)

    return df

import pandas as pd
import numpy as np
import re

# Function to clean parcel numbers by removing hyphens, non-numeric characters, and handling NaN values
def clean_parcel(parcel):
    """Clean parcel numbers by removing hyphens, non-numeric characters, and handling NaN values."""
    if pd.isna(parcel):
        return "Unknown"
    else:
        return re.sub(r'\D', '', parcel)  # Remove all non-numeric characters

# Function to validate the parcels based on length and content
def validate_parcel(parcel):
    """Validate the parcel based on length and content."""
    # Check if the parcel is a string and consists of exactly 9-10 digits
    if isinstance(parcel, str) and re.match(r'^\d{9,10}$', parcel):
        return parcel
    else:
        return "Unknown"

# Function to expand parcel numbers by handling complex cases like 'and', 'thru', semicolons, slashes, and other edge cases
def expand_parcel_numbers(parcel):
    """Expand parcel numbers by handling complex cases like 'and', 'thru', semicolons, slashes, and new edge cases."""
    if pd.isna(parcel) or not isinstance(parcel, str) or not parcel.strip():
        return ["Unknown"]

    # Replace 'and' with a comma and normalize 'thru'
    parcel = parcel.replace(' and ', ', ').replace('thru', ' thru ').replace(' to ', ' to ').replace('&', ', ')

    # Handle multiple semicolons or commas
    parcel = re.sub(r'[;,]+', ',', parcel)  # Replace multiple semicolons or commas with a single comma

    # Split by semicolons first to handle different parcel number groups
    groups = [group.strip() for group in parcel.split(';')]

    expanded_parcels = []

    for group in groups:
        # Split by commas for multiple parcels in the same group
        parts = [p.strip() for p in group.split(',')]

        base_parcel = None

        for part in parts:
            # Check if the part contains slashes (e.g., '3204008045/047') and handle accordingly
            if '/' in part:
                split_parts = part.split('/')
                base_parcel = split_parts[0]
                if len(split_parts) > 1:
                    for suffix in split_parts[1:]:
                        if suffix.isdigit():
                            base_prefix = base_parcel[:-len(suffix)]  # Adjust the prefix based on suffix length
                            expanded_parcels.append(validate_parcel(clean_parcel(base_prefix + suffix.zfill(3))))
                        else:
                            expanded_parcels.append("Unknown")
                    else:
                        expanded_parcels.append(validate_parcel(clean_parcel(base_parcel)))

            # Handle ranges with 'to' (e.g., '3123-014-900 to 916')
            elif ' to ' in part:
                try:
                    start, end = part.split(' to ')
                    start = start.strip()
                    end = end.strip()

                    if base_parcel is None:
                        base_parcel = start  # Base is the starting parcel number

                    base_prefix = '-'.join(base_parcel.split('-')[:-1]) + '-'

                    start_number = int(start.split('-')[-1])
                    end_number = int(end)

                    # Expand the range and append the parcels
                    for i in range(start_number, end_number + 1):
                        expanded_parcels.append(validate_parcel(clean_parcel(base_prefix + f"{i:03d}")))
                except Exception:
                    expanded_parcels.append("Unknown")  # Handle invalid range format

            # Handle ranges with 'thru' (e.g., '3203-018-064 thru -071')
            elif 'thru' in part:
                try:
                    start, end = part.split('thru')
                    start = start.strip()
                    end = end.strip().strip('-')

                    if base_parcel is None:
                        base_parcel = start  # Base is the starting parcel number

                    base_prefix = '-'.join(base_parcel.split('-')[:-1]) + '-'

                    start_number = int(start.split('-')[-1])
                    end_number = int(end.split('-')[-1])

                    # Expand the range and append the parcels
                    for i in range(start_number, end_number + 1):
                        expanded_parcels.append(validate_parcel(clean_parcel(base_prefix + f"{i:03d}")))
                except Exception:
                    expanded_parcels.append("Unknown")  # Handle invalid range format

            # Handle full parcel numbers (e.g., '3219-018-01')
            elif part.count('-') == 2:
                base_parcel = part
                expanded_parcels.append(validate_parcel(clean_parcel(base_parcel)))

            # Handle comma-separated extensions (e.g., '02' from '3219-018-01')
            elif base_parcel:
                base_prefix = '-'.join(base_parcel.split('-')[:-1]) + '-'
                full_parcel = validate_parcel(clean_parcel(base_prefix + part.strip('-')))
                expanded_parcels.append(full_parcel)

            else:
                expanded_parcels.append("Unknown")  # Handle cases where no base parcel is found

    return expanded_parcels

# Function to process the DataFrame and expand the parcel numbers
def process_parcel_data(df):
    """Process the DataFrame and expand the parcel numbers."""
    if 'Location Parcel Number' not in df.columns:
        raise ValueError("Column 'Location Parcel Number' is missing in the DataFrame.")

    # Apply the function to expand parcel numbers
    df['Expanded Parcels'] = df['Location Parcel Number'].apply(expand_parcel_numbers)

    # Find the maximum number of parcels in any row
    max_columns = df['Expanded Parcels'].apply(len).max()

    # Create the new columns dynamically based on the max number of parcels
    parcel_columns = pd.DataFrame(df['Expanded Parcels'].to_list(), columns=[f'Parcel_{i+1}' for i in range(max_columns)])

    # Replace any NaN or None values in the DataFrame with "Unknown"
    parcel_columns.fillna("Unknown", inplace=True)

    # Concatenate the new columns back to the original DataFrame
    df = pd.concat([df, parcel_columns], axis=1)

    # Drop the 'Expanded Parcels' and original 'Location Parcel Number' column
    df.drop(['Expanded Parcels', 'Location Parcel Number'], axis=1, inplace=True)

    return df

# Function to combine parcel columns into a single 'Location Parcel Number' column
def combine_parcels(df, max_length=50):
    parcel_columns = [col for col in df.columns if col.startswith('Parcel_')]
    df['Location Parcel Number'] = df[parcel_columns].apply(
        lambda row: ', '.join(row[row != 'Unknown'].astype(str)), axis=1)

    # Truncate if necessary
    df['Location Parcel Number'] = df['Location Parcel Number'].apply(
        lambda val: val[:max_length] if len(val) > max_length else val
    )
    
    # replace any None values with 'Unknown'
    df['Location Parcel Number'] = df['Location Parcel Number'].replace({None: 'Unknown'})
    df['Location Parcel Number'] = df['Location Parcel Number'].replace({'': 'Unknown'})

    # Drop extra Parcel columns
    df.drop(parcel_columns, axis=1, inplace=True)
    
    return df

# Function to execute the full process on a given DataFrame
def run_parcel_expansion(data):
    """Main function to run the parcel expansion process."""
    # Ensure data contains 'SCH Number' and 'Location Parcel Number'
    if data.empty:
        raise ValueError("Data is empty.")

    if not set(['SCH Number', 'Location Parcel Number']).issubset(data.columns):
        raise ValueError("Data must contain 'SCH Number' and 'Location Parcel Number' columns")

    # Process the DataFrame to expand parcel numbers
    df_cleaned = process_parcel_data(data)

    # Combine the expanded parcels back into a single column
    df_cleaned = combine_parcels(df_cleaned)

    return df_cleaned

def generate_entry_id_and_date_gathered(df, sch_column='SCH Number', month_column='Received_month', day_column='Received_day', year_column='Received_year'):
    """
    Generate a unique 'entry_id' by concatenating the SCH Number, Received_month, Received_day, and Received_year,
    and add a 'date_gathered' column with the current date (month, day, and year).

    Args:
        df (pd.DataFrame): DataFrame containing the relevant columns for SCH Number and Received date parts.
        sch_column (str): Column name for SCH Number. Default is 'SCH Number'.
        month_column (str): Column name for the received month. Default is 'Received_month'.
        day_column (str): Column name for the received day. Default is 'Received_day'.
        year_column (str): Column name for the received year. Default is 'Received_year'.
    
    Returns:
        pd.DataFrame: The input DataFrame with new 'entry_id' and 'date_gathered' columns.
    """
    # Generate 'entry_id' by concatenating the relevant columns
    df.loc[:, 'entry_id'] = (
        df[sch_column].astype(str) + 
        df[month_column].astype(str) + 
        df[day_column].astype(str) + 
        df[year_column].astype(str)
    )
    
    # Get the current date and format it as month, day, and year
    current_date = datetime.now()
    df.loc[:, 'date_gathered'] = current_date.strftime('%Y-%m-%d')  # Adds date in 'YYYY-MM-DD' format

    return df

def reorder_filtered_columns(data):
    
    df_filtered = data[['entry_id', 'SCH Number', 'Lead Agency Title', 'Document Title', 
                           'Document Type', 'Received', 'Posted', 'Document Description', 
                           'Cities', 'Counties', 'Location Cross Streets', 
                           'Location Total Acres', 'NOC Project Issues', 
                           'NOC Public Review Start Date', 'NOC Public Review End Date', 
                           'NOE Exempt Status', 'NOE Exempt Citation', 'NOE Reasons for Exemption', 
                           'NOD Agency', 'NOD Approved By Lead Agency', 'NOD Approved Date', 
                           'NOD Significant Environmental Impact', 
                           'NOD Environmental Impact Report Prepared', 
                           'NOD Negative Declaration Prepared', 'NOD Other Document Type', 
                           'NOD Mitigation Measures', 'NOD Mitigation Reporting Or Monitoring Plan', 
                           'NOD Statement Of Overriding Considerations Adopted', 
                           'NOD Findings Made Pursuant', 'NOD Final EIR Available Location', 
                           'date_gathered', 'Location Parcel Number']]
    
    # drop duplicate entry_id
    df_filtered.drop_duplicates(subset=['entry_id'], inplace=True)
    
    return df_filtered

In [33]:
import pandas as pd
# List of columns to keep
KEEPS = [
        'SCH Number', 'Lead Agency Title', 'Document Title', 'Document Type', 'Received', 
        'Posted', 'Document Description', 'Cities', 'Counties', 'Location Cross Streets', 
        'Location Parcel Number', 'Location Total Acres', 'NOC Project Issues', 
        'NOC Public Review Start Date', 'NOC Public Review End Date', 'NOE Exempt Status',
        'NOE Exempt Citation', 'NOE Reasons for Exemption', 'NOD Agency', 
        'NOD Approved By Lead Agency', 'NOD Approved Date', 'NOD Significant Environmental Impact', 
        'NOD Environmental Impact Report Prepared', 'NOD Negative Declaration Prepared', 
        'NOD Other Document Type', 'NOD Mitigation Measures', 
        'NOD Mitigation Reporting Or Monitoring Plan', 
        'NOD Statement Of Overriding Considerations Adopted', 
        'NOD Findings Made Pursuant', 'NOD Final EIR Available Location'
    ]

df = pd.read_csv('CEQA Documents.csv', encoding='ISO-8859-1')
pd.set_option('display.max_columns', None)

df_filtered = df.loc[:, KEEPS]
df_filtered = df_filtered.fillna('Unknown')
df_filtered = split_received_date(df_filtered, 'Received')
df_filtered = generate_entry_id_and_date_gathered(df_filtered)
df_expanded = run_parcel_expansion(df_filtered)
df_reorder = reorder_filtered_columns(df_expanded)

C:\Users\Julian Sotelo\AppData\Local\Temp\ipykernel_37384\1587069691.py:428: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.drop_duplicates(subset=['entry_id'], inplace=True)


In [34]:
# Check for duplicate entry_id values in the DataFrame
duplicates = df_reorder[df_reorder.duplicated(subset='entry_id', keep=False)]
print(duplicates)

Empty DataFrame
Columns: [entry_id, SCH Number, Lead Agency Title, Document Title, Document Type, Received, Posted, Document Description, Cities, Counties, Location Cross Streets, Location Total Acres, NOC Project Issues, NOC Public Review Start Date, NOC Public Review End Date, NOE Exempt Status, NOE Exempt Citation, NOE Reasons for Exemption, NOD Agency, NOD Approved By Lead Agency, NOD Approved Date, NOD Significant Environmental Impact, NOD Environmental Impact Report Prepared, NOD Negative Declaration Prepared, NOD Other Document Type, NOD Mitigation Measures, NOD Mitigation Reporting Or Monitoring Plan, NOD Statement Of Overriding Considerations Adopted, NOD Findings Made Pursuant, NOD Final EIR Available Location, date_gathered, Location Parcel Number]
Index: []
